# Parsing a book reader database with SQL

## Project description

The purpose of the project is to analyze the database of a large service for reading books by subscription.

Tasks:
1. Calculate how many books have been published since January 1, 2000;
2. For each book, count the number of reviews and the average rating;
3. Determine the publishing agency that has released the largest number of books thicker than 50 pages (to exclude brochures from the analysis);
5. Determine the author with the highest average book rating, considering only books with 50 or more ratings;
6. Calculate the average number of reviews from users who have given more than 50 ratings.

### Data Description

1. The books table contains data about books:

* book_id — book identifier;
* author_id - author ID;
* title — title of the book;
* num_pages - number of pages;
* publication_date — publication date of the book;
* publisher_id - publisher ID.

2. The authors table contains data about the authors:

* author_id - author ID;
* author - the name of the author.

3. The publishers table contains data about publishers:

* publisher_id - publisher identifier;
* publisher — publisher name;

4. The ratings table contains data on user ratings of books:

* rating_id — rating identifier;
* book_id — book identifier;
* username - the name of the user who left the rating;
* rating — book rating.

5. The reviews table contains data about user reviews of books:

* review_id — review ID;
* book_id — book identifier;
* username - the name of the user who wrote the review;
* text - review text.

Let's prepare the connection to the database.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
# set parameters
db_config = {'user': '***', # user 
 'pwd': '***', # pass
 'host': '***',
 'port': 1234, # connecting port
 'db': '***'} # database name
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# save the connector
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Task 1

Let's count how many books came out after January 1, 2000.

In [6]:
query = '''
            SELECT COUNT(book_id) AS books_number
            FROM books
            WHERE CAST(publication_date AS date) > '2000-01-01'
        '''
pd.io.sql.read_sql(query, con = engine)

,books_number
0,819


Conclusion: since January 1, 2000, 819 books have been published.

## Task 2

For each book, you need to count the number of reviews and the average rating.

In [7]:
query = ''' SELECT b.title AS title,
                   b.book_id AS id,
                   rev.r_count AS reviews,
                   rat.avg_rat AS avg_rating
            FROM books AS b
            LEFT JOIN 
                (SELECT book_id,
                        AVG(rating) AS avg_rat
                 FROM ratings
                 GROUP BY book_id
                ) AS rat ON rat.book_id = b.book_id
            LEFT JOIN
                (SELECT book_id,
                        COUNT(review_id) AS r_count
                 FROM reviews
                 GROUP BY book_id
                ) AS rev ON rev.book_id = b.book_id

        '''
pd.set_option('display.max_columns', None)  
pd.options.display.max_colwidth = 150
pd.io.sql.read_sql(query, con = engine)



,title,id,reviews,avg_rating
0,The Body in the Library (Miss Marple #3),652,2.0,4.500000
1,Galápagos,273,2.0,4.500000
2,A Tree Grows in Brooklyn,51,5.0,4.250000
3,Undaunted Courage: The Pioneering First Mission to Explore America's Wild Frontier,951,2.0,4.000000
4,The Prophet,839,4.0,4.285714
...,...,...,...,...
995,Alice in Wonderland,64,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),55,2.0,5.000000
997,Christine,148,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),790,2.0,3.500000


Top 3 books by number of ratings:

1. Twilight - 1120
2. Hobbit - 528
3. Catcher in the Rye - 516.

## Task 3

Determine the publishing house that produced the largest number of books thicker than 50 pages (to exclude brochures from the analysis).

In [8]:
query = '''
            SELECT p.publisher AS publisher_title,
                   COUNT(b.book_id) AS books_count
            FROM publishers AS p
            JOIN books AS b ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY books_count DESC
            LIMIT 1
            
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_title,books_count
0,Penguin Books,42


The publisher with the most books published is Penguin Books, with 42 books.

## Task 4

Determine the author with the highest average book rating, considering only books with 50 or more ratings.

In [12]:
query = '''
           SELECT i.author,
                  AVG(i.avg_rating) AS books_rating
           FROM
               (SELECT a.author AS author, 
                       AVG(rat.rating) AS avg_rating
                FROM books AS b
                LEFT JOIN authors AS a ON b.author_id = a.author_id
                LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
                GROUP BY b.book_id, a.author_id
                HAVING COUNT(rat.rating_id) >=50) AS i
                
           GROUP BY i.author
           ORDER BY books_rating DESC
           LIMIT 1

        '''
pd.io.sql.read_sql(query, con = engine)


,author,books_rating
0,J.K. Rowling/Mary GrandPré,4.283844


The author with the highest average rating among books with 50 or more ratings is J.K. Rowling/Mary GrandPré (4.29).

## Task 5

Calculate the average number of reviews from users who have given more than 50 ratings.

In [10]:
query = '''
            SELECT ROUND(AVG(i.review_number)) AS avg_reviews_count
            FROM
            (SELECT DISTINCT(r.username),
                   COUNT (DISTINCT(r.review_id)) AS review_number,
                   COUNT (DISTINCT(rat.rating_id)) AS rating_number
            FROM reviews AS r
            JOIN ratings AS rat ON r.username = rat.username
            GROUP BY r.username
            HAVING COUNT(DISTINCT(rat.rating_id)) > 50) AS i
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews_count
0,24.0


The average number of reviews from users who have given more than 50 ratings is 24 reviews.